<a href="https://colab.research.google.com/github/GwiHwan-Go/Detect_SC/blob/main/Data_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 수집한 데이터를 분석해 봅시다.

##### 본 스크립트의 코드들은 3-4분 정도의 연산 시간을 필요로 합니다.
##### 실질적인 데이터 분석 및 시각화는 Data_Visualization 스크립트를 참고해주세요.

참고 자료 : [딥러닝을 이용한 자연어 처리 입문](!https://wikidocs.net/92961), [SoyNLP 공식 문서](!https://github.com/lovit/soynlp)

# 준비 사항

본 파일은 체험용 코드입니다.

실제로 본 파일에 사용된 데이터는 240만 개의 게시글이며, 

여기서는 간단한 체험을 위해서 체험용 데이터를 제공합니다. 

또한, 미리 훈련시킨 soyNLP Tokenizer 를 제공하며, 

다운로드 방법은 아래에서 설명할 것입니다.

(수정중)

In [1]:
!pip install soynlp
!pip install konlpy
import pandas as pd

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## 제공하는 데이터 설명

1. 데이터 설명 : DCinside 라는 커뮤니티에서 우울증 갤러리라는 곳의 게시물들의 '제목'들.
2. 데이터 오버뷰

  - 데이터 크기 : 2,470,236 개

  - 데이터 피쳐 : title (텍스트 데이터), author (작성자 id), date (작성 날짜)
  
3. 텍스트 데이터 : 미리 학습시킨 토크나이저로 토큰화한 데이터로 ' '로 구분되어 있음.




In [3]:
# 데이터 다운로드, 파일명은 sns_data.csv 이다.
file_name = 'sns_data.csv'
!wget --load-cookies ~/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies ~/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1-4qx3IRyXUtb_QSItiWyIHZIo9hcBttg' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1-4qx3IRyXUtb_QSItiWyIHZIo9hcBttg" -O $file_name && rm -rf ~/cookies.txt


--2022-10-30 15:57:09--  https://docs.google.com/uc?export=download&confirm=t&id=1-4qx3IRyXUtb_QSItiWyIHZIo9hcBttg
Resolving docs.google.com (docs.google.com)... 142.251.16.113, 142.251.16.139, 142.251.16.102, ...
Connecting to docs.google.com (docs.google.com)|142.251.16.113|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-0o-30-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/tc6oojfhp57aaj44tet8b765iij63j1u/1667145375000/15740322463873228959/*/1-4qx3IRyXUtb_QSItiWyIHZIo9hcBttg?e=download&uuid=69a5a44e-2352-48ed-b198-260deeb7de14 [following]
--2022-10-30 15:57:10--  https://doc-0o-30-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/tc6oojfhp57aaj44tet8b765iij63j1u/1667145375000/15740322463873228959/*/1-4qx3IRyXUtb_QSItiWyIHZIo9hcBttg?e=download&uuid=69a5a44e-2352-48ed-b198-260deeb7de14
Resolving doc-0o-30-docs.googleusercontent.com (doc-0o-30-docs.googleusercontent.com)... 142.251.163.132,

In [4]:
# 데이터 살펴보기

data = pd.read_csv(file_name, low_memory=False, index_col=0) 
first_date, last_date = data.loc[0,'date'], data.loc[data.shape[0]-1,'date']
print("수집한 게시물 범위 :", first_date, "~", last_date, "\n" \
      "수집된 데이터 크기 :", data.shape[0], "\n수집한 데이터 features :", data.columns)
data.tail()

수집한 게시물 범위 : 2021-03-03 ~ 2022-10-27 
수집된 데이터 크기 : 2470236 
수집한 데이터 features : Index(['title', 'author', 'date'], dtype='object')


,title,author,date
2470231,나 병신 ...,16125,2022-10-27
2470232,나고 닉 글 올리 규 싶 읃데,12779,2022-10-27
2470233,씨발 남자 직원 음식 깔아 주,5291,2022-10-27
2470234,나도 근데 아이디 두 번 바꿈,15378,2022-10-27
2470235,알바 바보 쉨 ㅋㅋㅋ 나도 차단 해봐라,16125,2022-10-27


In [7]:
## 혹시 모를, nan 값을 제거해주자.
print(f"NAN 제거 전 데이터 수 = {data.shape}")
data = data[data.title.isna()==False]
data = data.reset_index(drop=True)
print(f"NAN 제거 후 데이터 수 = {data.shape}")

NAN 제거 전 데이터 수 = (2469803, 3)
NAN 제거 후 데이터 수 = (2469803, 3)


### 커스텀 토크나이저 만들기 w/ SoyNLP

기존의 토크나이저(konlpy, hanspell 등)들은 대부분 뉴스, 위키백과 등을 학습하여 만들어졌다. 

하지만, 우리가 오늘 다룰 데이터는 SNS 데이터로 각종 **신조어, 비표준어**가 난무한다.

기존의 토크나이저는 이와 같은 데이터를 잘 다루지 못하기 때문에,

[SoyNLP](!https://github.com/lovit/soynlp#tokenizer) 에서 제시한 비지도 학습 기반 토크나이저를 채택하였다.

이 토크나이저는 동질의 문서의 구조를 학습해서 글자 간의 __응집력__을 통해서 단어를 구분하는 알고리즘으로 설계되었다. 

자세한 부분은 [공식 문서](!https://github.com/lovit/soynlp#tokenizer)를 참고 해보자.

SoyNLP Tokenizer는 학습시키는데, 30분 정도 걸리기 때문에,

여기서는 미리 학습시킨 Tokenizer 모델을 제공한다.

직접 학습시키고 싶다면, 아래 코드를 돌려보자. 

더 자세한 내용은 [여기](!https://github.com/lovit/soynlp#tokenizer)에서 확인할 수 있다.

```python
    from soynlp.word import WordExtractor
    word_extractor = WordExtractor()
    word_extractor.train(data[data.title.isna()==False].title)
    word_score_table = word_extractor.extract()
```

In [8]:
extractor_name = 'soy_wordExtractor.model'
!wget --load-cookies ~/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies ~/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1-RrET_of1A3NVWWYnr3PMR-apTZ9dB7J' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1-RrET_of1A3NVWWYnr3PMR-apTZ9dB7J" -O $extractor_name && rm -rf ~/cookies.txt


--2022-10-30 16:07:04--  https://docs.google.com/uc?export=download&confirm=&id=1-RrET_of1A3NVWWYnr3PMR-apTZ9dB7J
Resolving docs.google.com (docs.google.com)... 172.253.63.102, 172.253.63.113, 172.253.63.100, ...
Connecting to docs.google.com (docs.google.com)|172.253.63.102|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-0o-30-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/amoukcgj7lmivekl7l96pksi2g0t5khv/1667145975000/15740322463873228959/*/1-RrET_of1A3NVWWYnr3PMR-apTZ9dB7J?e=download&uuid=07199435-1b62-4bf1-822c-3106dadee18c [following]
--2022-10-30 16:07:15--  https://doc-0o-30-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/amoukcgj7lmivekl7l96pksi2g0t5khv/1667145975000/15740322463873228959/*/1-RrET_of1A3NVWWYnr3PMR-apTZ9dB7J?e=download&uuid=07199435-1b62-4bf1-822c-3106dadee18c
Resolving doc-0o-30-docs.googleusercontent.com (doc-0o-30-docs.googleusercontent.com)... 142.251.163.132, 

In [9]:
import math
from soynlp.word import WordExtractor
from soynlp.tokenizer import LTokenizer

## Word_Extractor param 설정 및 로드
word_extractor = WordExtractor(min_frequency=10,
                               min_cohesion_forward=5.0,
                               min_right_branching_entropy=0.0)
word_extractor.load(extractor_name)

## Cohesion 과 Entropy 계산해서 tokenizer 에 필요한 scores 계산
scores = word_extractor.word_scores()
scores = {key : (scores[key].cohesion_forward * math.exp(scores[key].right_branching_entropy)) \
          for key in scores.keys()}

## tokenizer 생성
tokenizer = LTokenizer(scores=scores)

all cohesion probabilities was computed. # words = 480337
all branching entropies was computed # words = 523918
all accessor variety was computed # words = 523918


### 커스텀 토크나이저

본 분석에서 이용한 커스텀 토크나이저를 소개한다.

SoyNLP 토크나이저는 품사 태깅을 하지 못해서 Okt 모듈로 품사 태깅을 한 후에

조사를 제외해주었다.

In [10]:
## SoyNLP 사용 예 -> SoyNLP(tokenizer) -> okt pos tagger -> 불용어 제거
from konlpy.tag import Okt
from soynlp.normalizer import *

def preprocess_tokenizer(sentence) : 

  pos_tagger = Okt()

  return [word[0]
          for word in pos_tagger.pos(' '.join(tokenizer.tokenize(sentence)))
          if word[1] != 'Josa'] # 조사는 분석에서 제외


In [13]:
## 문장을 넣어서 토크나이저의 성능을 체크해보자.
play_sentences= "야이개색야"
preprocess_tokenizer(play_sentences)

['야', '이', '개색']

## PMI 분석

출처 : [SoyNLP 공식문서](!https://github.com/lovit/soynlp/blob/master/tutorials/pmi_usage.ipynb)

이제 마지막 단계이다.

단어들 간의 연관관계를 분석해서 자살 관련 정보 파악에 도움이 되는 __다른 단어__들은 무엇이 있는 지 알아보자.

PMI 분석은

(word, contexts) 혹은 (input, outputs)의 상관성을 측정하는 방법으로, pmi 값이 클수록 두 변수의 상관성이 크다.

이를 위해서 특정 단어와 그 주위 단어들을 저장하는 행렬인 Word-Contexts Matrix를 생성한다.

In [14]:
# 1분 30초 걸림
from soynlp.vectorizer import sent_to_word_contexts_matrix
from scipy import sparse
import pickle
from soynlp.word import pmi as pmi_func

x, idx2vocab = sent_to_word_contexts_matrix(
    data.title,
    windows=3,
    min_tf=10,
    tokenizer= lambda x:x.split(), # (default) lambda x:x.split(),
    dynamic_weight=False,
    verbose=True
)

pmi, px, py = pmi_func(
    x,
    min_pmi = 0,
    alpha = 0.0,
    beta = 0.75
)

Create (word, contexts) matrix
  - counting word frequency from 2469802 sents, mem=1.771 Gb
  - scanning (word, context) pairs from 2469802 sents, mem=2.756 Gb
  - (word, context) matrix was constructed. shape = (49771, 49771)                    
  - done


### 결과

이제 우울증 갤러리에서 주로 얘기되는 단어들 간의 연결관계를 알 수 있다.

- Context Words : 비슷한 문맥을 가지는 단어.

- Similar Words : 문맥이 비슷한 단어.

target 변수에 알고싶은 단어를 넣어서 더 자세히 알아보자.

In [15]:
## 특정 단어['살고 싶다']와 함께 등장한 단어들 상위 10개##
target = '자살'
howmany = 10

vocab2idx = {vocab:idx for idx, vocab in enumerate(idx2vocab)}
query = vocab2idx[target]

submatrix = pmi[query,:].tocsr() # get the row of query
contexts = submatrix.nonzero()[1] # nonzero() return (rows, columns)
pmi_i = submatrix.data

most_relateds = [(idx, pmi_ij) for idx, pmi_ij in zip(contexts, pmi_i)]
most_relateds = sorted(most_relateds, key=lambda x:-x[1])[:howmany]
most_relateds = [(idx2vocab[idx], pmi_ij) for idx, pmi_ij in most_relateds]

from pprint import pprint
pprint(most_relateds)

[('도피', 6.844622458591964),
 ('삵', 6.031369397834147),
 ('살고싶은데', 5.772712722570554),
 ('뽁뽁', 5.685233690813876),
 ('했데', 5.524714801301454),
 ('좆피', 5.288600593862374),
 ('자살', 5.274353861018971),
 ('머애', 5.262282104003921),
 ('은부', 5.249461278984446),
 ('대왕고래', 5.23685594024716)]


In [16]:
## 특정 단어['우울']과 컨텍스트가 가장 비슷한 단어 상위 10개##

from soynlp.utils import most_similar

most_similar('우울', pmi, vocab2idx, idx2vocab)

[('심심', 0.22781712789543906),
 ('불안', 0.20654552217405342),
 ('우울함', 0.16989673506620928),
 ('행복', 0.16271023927659367),
 ('우울증', 0.1625480654125695),
 ('피곤', 0.15479783161764749),
 ('우울해', 0.14304488688384542),
 ('우울하다', 0.1355665302590453),
 ('불행', 0.13496097718192268),
 ('우울한', 0.13324270584997489)]

# 텍스트 분석 끝!
